## Imports

In [12]:
from collections import OrderedDict
from bs4 import BeautifulSoup
from fuzzywuzzy import process
import altair as alt
import pandas as pd

from selenium.common.exceptions import TimeoutException
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# Scraping w/ Selenium 

In [123]:
zack_url = "https://scholar.google.com/citations?user=X7FY3wUAAAAJ&hl=en&oi=ao"
hinton_url = "https://scholar.google.com/citations?user=JicYPdAAAAAJ&hl=en&oi=ao"
malcolm_url = "https://scholar.google.de/citations?user=bcO-7KwAAAAJ&hl=en&oi=ao"
tarnas_url = "https://scholar.google.com/citations?hl=en&user=H9AJzHMAAAAJ&view_op=list_works),"
allegra_url = "https://scholar.google.com/citations?user=CfnwDC4AAAAJ&hl=en"
kessler_url = "https://scholar.google.com/citations?user=EicYvbwAAAAJ&hl=en&inst=5746887945952177237&oi=ao)"
foucault_url = "https://scholar.google.com/citations?user=AKqYlxMAAAAJ&hl=en&inst=5746887945952177237&oi=ao)"
bourdieu_url = "https://scholar.google.com/citations?user=d_lp40IAAAAJ&hl=en&inst=5746887945952177237&oi=ao),"

In [144]:
options = ChromeOptions()
options.headless = True
driver = Chrome(options=options)

In [145]:
type(driver)

selenium.webdriver.chrome.webdriver.WebDriver

In [146]:
button_xpath = "/html/body/div/div[13]/div[2]/div/div[4]/form/div[2]/div/button"

In [147]:
%%time
# base_url = malcolm_url
# base_url = zack_url
# base_url = hinton_url
# base_url = tarnas_url
# base_url = allegra_url
base_url = kessler_url
# base_url = foucault_url
# base_url = bourdieu_url
driver.get(base_url)
count = 0

while True:  
    try:
        button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
        button.click()
        count += 1
        print(f"click number: {count}")
        
    except TimeoutException:
        print('not clickable')
        break
        
html = driver.page_source
base_page = BeautifulSoup(html, "lxml")
driver.close()

links = []
for td in base_page.find_all("td", attrs={"class": "gsc_a_t"}):
    link = td.find("a").get("data-href")
    full_link = base_url.split("citations?")[0] + link
    links.append(full_link)
print("number of publication links found: ", len(links))

click number: 1
click number: 2
click number: 3
click number: 4
click number: 5
click number: 6
click number: 7
click number: 8
click number: 9
click number: 10
click number: 11
click number: 12
click number: 13
click number: 14
click number: 15
click number: 16
click number: 17
click number: 18
click number: 19
not clickable
number of publication links found:  1900
Wall time: 24.9 s


In [148]:
type(base_page)

bs4.BeautifulSoup

# Get data w/ BeautifulSoup

### Get author name

In [149]:
page_title = base_page.find('title').string
author_name = page_title.split(' - ')[0]
author_name

'Ronald C Kessler'

### Get current role

In [150]:
temp = []
for x in base_page.find_all("div", attrs={"class": "gsc_prf_il"}):
    temp.append(x.text)
    
temp[0]

'Professor of Health Care Policy, Harvard Medical School'

### Exclude duplicate publications

In [151]:
pubs_data = []
for tr in base_page.find_all("tr", attrs={"class": "gsc_a_tr"}):
    # check if it contains an attribute specific to duplicates
    if tr.find_all("a")[1].has_attr("data-eud"):
        continue
    
    else:
        td1, td2, td3 = tr.find_all("td")
        
        authors = td1.find("div").contents
        citations = td2.find("a").contents
        year = td3.find("span").contents
        
        if len(citations) < 1:
            citations = None
        else:
            citations = int(citations[0])
            
        if len(year) < 1:
            year = None
        else:
            year = int(year[0])
        
        data = {
            "authors": authors, 
            "citations": citations, 
            "year": year
        }
        pubs_data.append(data)
    
print(len(pubs_data))

1900


In [152]:
pubs_data[:10]

[{'authors': ['RC Kessler, P Berglund, O Demler, R Jin, KR Merikangas, EE Walters'],
  'citations': 25851,
  'year': 2005},
 {'authors': ['RC Kessler, KA McGonagle, S Zhao, CB Nelson, M Hughes, S Eshleman, ...'],
  'citations': 16763,
  'year': 1994},
 {'authors': ['RC Kessler, WT Chiu, O Demler, EE Walters'],
  'citations': 12858,
  'year': 2005},
 {'authors': ['RC Kessler, A Sonnega, E Bromet, M Hughes, CB Nelson'],
  'citations': 12825,
  'year': 1995},
 {'authors': ['DM Eisenberg, RB Davis, SL Ettner, S Appel, S Wilkey, M Van Rompay, ...'],
  'citations': 9133,
  'year': 1998},
 {'authors': ['RC Kessler, G Andrews, LJ Colpe, E Hiripi, DK Mroczek, SLT Normand, ...'],
  'citations': 6031,
  'year': 2002},
 {'authors': ['DM Eisenberg, RC Kessler, C Foster, FE Norlock, DR Calkins, ...'],
  'citations': 5932,
  'year': 1993},
 {'authors': ['JI Hudson, E Hiripi, HG Pope Jr, RC Kessler'],
  'citations': 4690,
  'year': 2007},
 {'authors': ['RC Kessler, L Adler, R Barkley, J Biederman, CK 

### Get total citations

In [153]:
count = 0
for pub in pubs_data:
    if pub["citations"]:
        count += pub["citations"]
    
print(count)

401630


### Get citation counts for each paper

In [24]:
# get citations for each paper 
citations_lis = []
for td in base_page.find_all("td", attrs={"class": "gsc_a_c"}):
    citation = td.find("a").contents
    citations_lis.append(citation)

print(citations_lis[:3])
print(len(citations_lis))

[['79511'], ['79418'], ['59196']]
1007


In [25]:
citations_lis = [int(c[0]) if len(c)==1 else 0 for c in citations_lis]
len(citations_lis)

1007

In [26]:
sum(citations_lis)

1416490

Weird that this different from Google's listed total citations for Geoffrey Hinton: 372718.

Seems they might be taking into account duplicate publications when counting total citations, but not when calculating the author's overall h-index, as you'll see later in the notebook.

### Get publication dates

In [14]:
# get years for each paper 
years_lis = []
for yr in base_page.find_all("span", attrs={"class": "gs_oph"}):
    years_lis.append(yr.text)

# remove the extras
# years_lis = [text for i,text in enumerate(years_lis) if i%2==0]
years_lis[:5]

[', 2005', ', 1994', ', 2005', ', 1995', ', 1998']

In [15]:
len(years_lis)

1757

There are missing values from the years list because sometimes that section is blank on the website, but the year is listed in the title of the paper section instead.

Also, it seems there are duplicate publications listed on some Google Scholar profiles. For example on Geoffrey Hinton's, "Layer Normalization" is listed three different times. Same authors, same journal (arXiv), but different citations counts.

Google is not taking this into account in there h-index calculations b/c I reproduced theirs by including duplicates.

In [16]:
# TODO: 

# would need to extract the dates from the publication title where date is missing from usual spot,
# and account for bogus years like "6" for "Williams, RI (1986). Learning internal representations by error propagation"
# and sometimes there's no date listed at all in the title either

# also just need to clean up the year strings (ex: ', 1986' --> 1986)

### Get co-authors for each publication

In [17]:
# clean up and split names
# author_lists = [names.split(', ') for names in authors_lis]

# print(len(author_lists))

In [18]:
# author_lists[:3]

### Get backup authors list

In [19]:
# get authors of each paper
backup_authors = []
for auth in base_page.find_all("div", attrs={"class": "gs_gray"}):
    backup_authors.append(auth.text)
    
# remove extra gs_gray classes found (the journal names)
backup_authors = [text for i,text in enumerate(backup_authors) if i%2==0]

# clean up and split names
backup_authors = [names.split(', ') for names in backup_authors]

In [20]:
backup_authors[:5]

[['RC Kessler',
  'P Berglund',
  'O Demler',
  'R Jin',
  'KR Merikangas',
  'EE Walters'],
 ['RC Kessler',
  'KA McGonagle',
  'S Zhao',
  'CB Nelson',
  'M Hughes',
  'S Eshleman',
  '...'],
 ['RC Kessler', 'WT Chiu', 'O Demler', 'EE Walters'],
 ['RC Kessler', 'A Sonnega', 'E Bromet', 'M Hughes', 'CB Nelson'],
 ['DM Eisenberg',
  'RB Davis',
  'SL Ettner',
  'S Appel',
  'S Wilkey',
  'M Van Rompay',
  '...']]

# Format data by year(to-do), co-author pos, & h-index

### Get co-author positions

In [21]:
# scary one-liner to convert int to ordinal representation (ex: 2 --> "2nd")
# https://stackoverflow.com/questions/9647202/ordinal-numbers-replacement
# ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

In [22]:
full_name_lis = author_name.split()
last_name = full_name_lis.pop()
initials = "".join([name[0] for name in full_name_lis])
shortened_auth_name = initials + " " + last_name
shortened_auth_name

'RC Kessler'

In [23]:
process.extractOne(shortened_auth_name, ['DS Scheirer', 'AH Scheirer'])

('DS Scheirer', 46)

In [24]:
print("author name: ", author_name)

def get_author_positions_lis(auth_name, auth_lists):
    ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])
    author_positions_lis = []

    for names in auth_lists:
#         print(names)
        try: 
#             match = process.extractOne(auth_name, names, score_cutoff=75)[0]
            matches = process.extract(auth_name, names, limit=2)
#             print(matches)
            if matches:
                # make sure they're not too far off origianl name
                if matches[0][1] > 74:
                    
                    if len(matches) == 2:
                        # if one's better, take that one
                        if matches[0][1] > matches[1][1]:
                            match = matches[0][0]
                
                        else:
                            full_name_lis = auth_name.split()
                            last_name = full_name_lis.pop()
                            initials = "".join([name[0] for name in full_name_lis])
                            shortened_auth_name = initials + " " + last_name
                            match = process.extractOne(shortened_auth_name, [name[0] for name in matches])[0]
                    else:
                        match = matches[0][0]
                else:
                    match = None
                    
        except TypeError: 
            match = None
#             matches = None
            print('no match')
        
#         print(match)
            
        if match:
            for i, author in enumerate(names):
                if author == match:

                    if i == len(names)-1 and i > 2:
                        author_positions_lis.append('last')
                    elif i > 4:
#                         author_positions_lis.append('6th or more')
                        author_positions_lis.append('≥ 6th')
                    else:
                        author_positions_lis.append(ordinal(i+1))
                    break
        else:
            if len(names) > 4:
#                 author_positions_lis.append('6th or more')
                author_positions_lis.append('≥ 6th')
            else:
                author_positions_lis.append(ordinal(len(names)))
            
    return author_positions_lis

# author_positions_lis = get_author_positions_lis(author_name, author_lists)
author_positions_lis = get_author_positions_lis(author_name, backup_authors)

print(author_positions_lis[:5])
print(len(author_positions_lis))

author name:  Ronald C Kessler
['1st', '1st', '1st', '1st', '≥ 6th']
1900


In [25]:
set(author_positions_lis)

{'1st', '2nd', '3rd', '4th', '5th', 'last', '≥ 6th'}

In [26]:
backup_authors[1]

['RC Kessler',
 'KA McGonagle',
 'S Zhao',
 'CB Nelson',
 'M Hughes',
 'S Eshleman',
 '...']

In [27]:
# needed to lower the cutoff to 75 for it to consider "JD Tarnas" a match for "Jesse Tarnas"
get_author_positions_lis(author_name, [backup_authors[1]])

['1st']

### Group citations by co-author position

In [28]:
def get_pos_dfs(pos_lis, num_lis):
    citations_positions_df = pd.DataFrame(list(zip(pos_lis, num_lis)), columns =['positions', 'citations']) 
    return dict(tuple(citations_positions_df.groupby('positions')))


dfs = get_pos_dfs(author_positions_lis, citations_lis)
dfs.keys()

dict_keys(['1st', '2nd', '3rd', '4th', '5th', 'last', '≥ 6th'])

In [29]:
dfs['1st'].head()

positions  citations
0       1st      25851
1       1st      16763
2       1st      12858
3       1st      12825
5       1st       6031

### Get h-indexes by co-author position

In [30]:
def get_hindexes_dict(dataframes):
    hindexes_dict = {}
    
    for k, df in dataframes.items():
        df.sort_values('citations')
        df.index += 1
        df = df.reset_index()
        df = df.query('citations >= index')
        # checking if there are no citations
        if df.shape[0] <= 0:
            hindexes_dict[k] = 0
        else:
            hindexes_dict[k] = df.shape[0]
        
    return hindexes_dict
        
hindexes_d = get_hindexes_dict(dfs)

In [31]:
# excluding co-author position w/ h-index of zero
hindexes_d

{'1st': 108,
 '2nd': 36,
 '3rd': 41,
 '4th': 23,
 '5th': 7,
 'last': 27,
 '≥ 6th': 50}

In [32]:
sum(hindexes_d.values())

292

In [33]:
# Regular h-index (same as Google's)
temp_df = pd.DataFrame({"citations": citations_lis})
temp_df.sort_values('citations')
temp_df.index += 1
temp_df = temp_df.reset_index()
temp_df = temp_df.query('citations >= index')
temp_df.shape[0]

292

In [34]:
pd.DataFrame({'position': list(hindexes_d.keys()), 'h-index': list(hindexes_d.values())}).set_index('position')

h-index
position         
1st           108
2nd            36
3rd            41
4th            23
5th             7
last           27
≥ 6th          50

# Format data for plotting

In [35]:
def get_counts_dicts(pos_lis, num_lis):
    d1 = {}
    d2 = {}

    for position, num in zip(pos_lis, num_lis):
        if position in d1:
            d1[position] += 1
            d2[position] += num
        else:
            d1[position] = 1
            d2[position] = num
    return d1, d2
        
author_positions, citations_by_author_position = get_counts_dicts(author_positions_lis, citations_lis)

print(f'author_positions: {author_positions}')
print(f'citations_by_author_position: {citations_by_author_position}')

author_positions: {'1st': 567, '≥ 6th': 588, '2nd': 256, 'last': 115, '3rd': 195, '4th': 115, '5th': 64}
citations_by_author_position: {'1st': 192216, '≥ 6th': 63207, '2nd': 46497, 'last': 30141, '3rd': 39043, '4th': 21810, '5th': 8716}


In [36]:
author_positions = OrderedDict(sorted(author_positions.items()))
citations = OrderedDict(sorted(citations_by_author_position.items()))

In [37]:
author_positions.move_to_end('last')
citations.move_to_end('last')

In [38]:
citations

OrderedDict([('1st', 192216),
             ('2nd', 46497),
             ('3rd', 39043),
             ('4th', 21810),
             ('5th', 8716),
             ('≥ 6th', 63207),
             ('last', 30141)])

In [39]:
sum(citations.values())

401630

In [40]:
author_positions

OrderedDict([('1st', 567),
             ('2nd', 256),
             ('3rd', 195),
             ('4th', 115),
             ('5th', 64),
             ('≥ 6th', 588),
             ('last', 115)])

In [41]:
print(author_positions)
print(citations)

OrderedDict([('1st', 567), ('2nd', 256), ('3rd', 195), ('4th', 115), ('5th', 64), ('≥ 6th', 588), ('last', 115)])
OrderedDict([('1st', 192216), ('2nd', 46497), ('3rd', 39043), ('4th', 21810), ('5th', 8716), ('≥ 6th', 63207), ('last', 30141)])


In [42]:
lis1 = list(citations.keys())
lis2 = list(citations.values())

In [43]:
lis1

['1st', '2nd', '3rd', '4th', '5th', '≥ 6th', 'last']

In [44]:
df = pd.DataFrame({
                "positions": lis1,
                "citations": lis2
            })
percents_df = df.copy()
percents_df.citations = (100 * df.citations / df.citations.sum()).round(0)
percents_df.citations = percents_df.citations / 100
# percents_df = percents_df.fillna(0)
percents_df

positions  citations
0       1st       0.48
1       2nd       0.12
2       3rd       0.10
3       4th       0.05
4       5th       0.02
5     ≥ 6th       0.16
6      last       0.08

# Plot w/ Altair

In [45]:
def bar_chart(df):
    return alt.Chart(df).mark_bar().encode(
        alt.X('citations', axis=alt.Axis(format='%')),
        alt.Y('positions', sort=None),
        color=alt.Color(
            "positions", 
            scale=alt.Scale(scheme="greenblue"), 
            legend=None
            )
        ).properties(
            title='portion of total citations by co-author position'
        ).configure_axisX(
            labelAngle=0
        ).configure_view(
            strokeWidth=0
        )


bar_chart(percents_df).display()

alt.Chart(...)